# Advent of Code 2022 - Day 11

In [28]:
import re
import math
import copy
import numpy as np

In [29]:
inp = open('input.txt','r').read()

In [30]:
d = [x.split("\n") for x in inp.split("\n\n")]

In [31]:
instructions = [[ int(x[0][-2]), #monkey number
[int(a) for a in re.findall(r'\d+',x[1])], #item worries
x[2].split("old ",1)[1].split(), #operation and number
int(re.findall(r'\d+',x[3])[0]), #divisible by
int(re.findall(r'\d+',x[4])[0]), #if true throw to monkey
int(re.findall(r'\d+',x[5])[0])] #if false throw to monkey
                for x in d]

In [32]:
monkey_item_list = [x[1] for x in instructions]
monkey_item_list = copy.deepcopy(monkey_item_list)

In [33]:
inspections = [0 for i in range(0,len(instructions))]

In [34]:
for rounds in range(1,21):
    for monkey,items,operation,divisor,true_throw,false_throw in instructions:
    #print('monkey is: ', monkey)
    
        for item in monkey_item_list[monkey]:
            inspections[monkey] += 1
            if operation[1] == 'old' :
                         op_number = item
            else:
                op_number = int(operation[1])
            if operation[0] == "*":
                level = item * op_number
            else:
                level = item + op_number

            #Passing item to new monkey
            level = math.trunc(level/3)            
            
            if level % divisor == 0:
                monkey_item_list[true_throw].append(level)
            else:
                monkey_item_list[false_throw].append(level)
        
        monkey_item_list[monkey] = []

In [35]:
# Part 1 answer
inspections.sort()
inspections[-1]*inspections[-2]

50830

## Part 2
Re-did the above using Numpy arrays because I thought that would help with scaling (?) .. Wasn't actually necessary and caused huge issues debugging when integer sizes got too big.

The hard bit was figuring out how to `keep your worry levels manageable`, as heavily hinted in the question.

Need to scale a number `x` -> `scaled_x` so that when `x % n == scaled_x % n` for n divisors. 

Use the `least common multiple` of our divisors. Then we can essentially 'chunk off' multiples of the divisors from our integers - in a way which will not affect whether it's divisible. 

In [36]:
inspections = [0 for i in range(0,len(instructions))]

In [37]:
monkey_item_list = [x[1] for x in instructions.copy()]
monkey_item_list = copy.deepcopy(monkey_item_list)

In [38]:
#set up operation flags
op_flags = [ [0,0,0,0] for x in instructions.copy()]

for monkey,items,operation,divisor,true_throw,false_throw in instructions:

    if operation[1] == 'old' and operation[0] == "*":
        op_flags[monkey][0] = 1
        
    elif operation[1] == 'old' and operation[0] == "+":
        op_flags[monkey][1] = 1

    elif operation[0] == "*":
        op_flags[monkey][2] = 1
    else:
        op_flags[monkey][3] = 1

In [39]:
def int_modify(x):
    try:
        return int(x)
    except Exception:
        return 0

In [54]:
scaling = math.lcm(*[monkey[3] for monkey in instructions])

In [41]:
for rounds in range(1,10001):
    #print("round is: ", rounds)
    for monkey,items,operation,divisor,true_throw,false_throw in instructions:
        
        items_array = np.array(monkey_item_list[monkey])

        inspections[monkey] += len(items_array)   

        scale_array = np.array([scale_factor]*len(items_array))
        items_array = np.mod((op_flags[monkey][0] * ( np.square(items_array.astype('object')))) + (op_flags[monkey][2] * ( items_array * int_modify(operation[1]) ).astype('int64') ) + (op_flags[monkey][3] * ( items_array + int_modify(operation[1])).astype('int64')),scaling).astype('int64')
        
        [monkey_item_list[true_throw].append(x) for x in (items_array[np.array( items_array % divisor == 0) ]).tolist()]
        [monkey_item_list[false_throw].append(x) for x in (items_array[np.array( ( (items_array % divisor == 0) == False)) ]).tolist()]
        
        monkey_item_list[monkey] = []
        

In [42]:
# Part 2 answer
inspections.sort()
inspections[-1]*inspections[-2]

14399640002